In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

Dataset

In [ ]:
# X_train = pd.read_csv('X_train.csv')
X_train = pd.read_csv('X_train_big_data.csv')
X_train = X_train.values

Euclidean Distance

In [ ]:
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1-x2)**2))

In [ ]:
def compute_distances(X,centroids):
    m = X.shape[0]
    n = len(centroids)
    distances = np.zeros((m, n), dtype=float)
    for i in range(m):
        for j in range(n):
            distances[i][j] = euclidean_distance(X[i], centroids[j])
    return distances

In [ ]:
def update_centroids(X, cluster_labels):
    n = len(set(cluster_labels))
    new_centroids = [] 

    for i in range(n):
        cluster_point = X[cluster_labels == i]
        centroids = np.mean(cluster_point, axis=0)
        new_centroids.append(centroids)
    return new_centroids

In [ ]:
def plot(X, centroids, cluster_labels):
    centroids_2d = centroids[:, :2] 
    plt.figure(figsize=(10, 6))

    # Lặp qua từng cụm để vẽ các điểm
    for cluster_id in np.unique(cluster_labels):
        cluster_points = X[cluster_labels == cluster_id]  # Truy cập vào các điểm thuộc cụm

        # Đảm bảo không gặp lỗi khi truy cập vào chỉ số
        if cluster_points.ndim == 2:  # Kiểm tra nếu là ma trận 2D
            plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'Cụm {cluster_id}')

    # Hiển thị các tâm cụm
    plt.scatter(centroids_2d[:, 0], centroids_2d[:, 1], color='black', marker='x', s=200, label='Tâm cụm')

    # Thêm tiêu đề và nhãn trục
    plt.title('Trực quan hóa phân cụm và các tâm cụm (2 đặc trưng đầu tiên)')
    plt.xlabel('Đặc trưng 1')
    plt.ylabel('Đặc trưng 2')
    plt.legend()

    plt.show()

Model

In [ ]:
def model(X, k):
    np.random.seed(1)
    random_indices = np.random.choice(X.shape[0], size=k, replace=False)
    centroids = X[random_indices]
    cluster = np.zeros(X.shape[0])
    
    while True:
        # Tính khoảng cách từ các mẫu đến từng tâm cụm
        distances = compute_distances(X,centroids)
        
        # Xác định cụm gần nhất cho từng mẫu
        new_cluster = np.argmin(distances, axis=1)
        
        # Nếu các cụm không thay đổi thì dừng vòng lặp
        if np.array_equal(cluster, new_cluster):
            break
        
        # Cập nhật các chỉ số cụm
        cluster = new_cluster
        
        # Cập nhật các tâm cụm dựa trên các mẫu trong từng cụm
        centroids = update_centroids(X, cluster)

    return cluster, centroids
    

Main

In [ ]:
k = 4

start_time = time.time()
predict, centroids = model(X_train, k)
end_time = time.time()
print(f"Thời gian chạy mô hình tự cài đặt: {end_time - start_time:.4f} giây")

centroids = np.array(centroids)

print("Kết quả phân cụm:", predict)
print("Các tâm cụm:", centroids)
plot(X_train, centroids, predict)

Sklearn

In [ ]:
from sklearn.cluster import KMeans

# Khởi tạo mô hình KMeans với k = 4
start_time = time.time()
kmeans = KMeans(n_clusters=4, random_state=42)

# Huấn luyện mô hình
kmeans.fit(X_train)
end_time = time.time()

print(f"Thời gian chạy mô hình sklearn: {end_time - start_time:.4f} giây")

# Nhận kết quả phân cụm
clusters = kmeans.labels_
centroids = kmeans.cluster_centers_

print("Kết quả phân cụm:", clusters)
print("Các tâm cụm:", centroids)

plot(X_train, centroids, clusters)

In [ ]:
from sklearn.metrics import silhouette_score
manual_silhouette = silhouette_score(X_train, predict)
sklearn_silhouette = silhouette_score(X_train, clusters)
print("Silhouette score manual:", manual_silhouette)
print("Silhouette score sklearn:", sklearn_silhouette)